In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
%matplotlib inline

In [ ]:
x = np.linspace(0, 1, 1000)
y = np.sin(2 * math.pi * x)

In [ ]:
sample_size = 5
np.random.seed(123456)
sample_x = np.random.choice(x, size=sample_size, replace=False)
sample_y = np.sin(2 * math.pi * sample_x)

## Vanilla setting (explicit)

In [ ]:
def k_gaussian(x1, x2):
    # calculating gaussian kernel matrix
    # the output has shape (len(x2), len(x1))
    # entry at [i, j] position is given by k(x2[i], x1[j])
    
    # gaussian kernel hyperparameters - adjusts the distance between points and variance
    l = 0.1
    sigma = 1
    
    x1_matrix = np.tile(x1, len(x2)).reshape((len(x2), len(x1)))
    x2_matrix = np.tile(x2, len(x1)).reshape((len(x1), len(x2))).transpose()
    
    k_matrix = np.exp(-(x1_matrix - x2_matrix) ** 2 / (2 * l * l)) * sigma ** 2
    
    return k_matrix

In [ ]:
def gp_posterior(sample_x, sample_y, x):
    # calculating posterior for gaussian processes
    # it is assumed that observations have some additional gaussian noise
    
    sigma_obs = 0.1  
    
    # Separately calculating matrix used to calculate both mean and variance
    K = np.dot(k_gaussian(sample_x, x),
               np.linalg.inv(k_gaussian(sample_x, sample_x) + np.eye(len(sample_x)) * sigma_obs ** 2)
              )
    
    mu = np.dot(K, sample_y)
    sigma = k_gaussian(x, x) - np.dot(K, k_gaussian(x, sample_x))
    
    return mu, sigma

In [ ]:
mu, sigma = gp_posterior(sample_x, sample_y, x)
std_1d = np.sqrt([sigma[i, i] for i in range(len(mu))])

In [ ]:
plt.plot(x, y, label="signal")
plt.plot(sample_x, sample_y, ".", color="r", label="obs")
plt.plot(x, mu, color="g", label="posterior")
plt.fill_between(x, mu - 2 * std_1d, mu + 2 * std_1d, color="g", alpha=0.5)

plt.title("True and recovered signals")
plt.legend()
plt.xlabel("x")
plt.show()

## Vanilla setting (sklearn)

Main difference so far: sklearn implementation actually estimates kernel hyperparameters through ML (becase of beta0=None), n_restarts specifies how many times ML is performed when starting from random points, noise in observed data is handled via alpha (is specifies the value to add along the main diagonal when inverting kernel matrix for observations)

(hopefully, it will be more stable when it comes to cholesky decomposition in later applications) 

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

gp = GaussianProcessRegressor(kernel=RBF(length_scale=0.1),
                              alpha=0.01,
                              n_restarts_optimizer=100,
                              normalize_y=False
                             )

gp.fit(sample_x.reshape(-1, 1), sample_y.reshape(-1, 1))

mu, std_1d = gp.predict(x.reshape(-1, 1), return_std=True)
std_1d = std_1d.reshape(-1, 1)

In [ ]:
plt.plot(x, y, label="signal")
plt.plot(sample_x, sample_y, ".", color="r", label="obs")
plt.plot(x, mu, color="g", label="posterior")
plt.fill_between(x, (mu - 2 * std_1d).reshape(-1), (mu + 2 * std_1d).reshape(-1), color="g", alpha=0.5)

plt.title("True and recovered signals")
plt.legend()
plt.xlabel("x")
plt.show()